In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report
from sklearn import tree, neighbors, metrics
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
train_data = pd.read_csv('./knights/Train_knight.csv')

X = train_data.drop(columns='knight', axis=1).values
y = train_data['knight'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = (X_train - X_train.mean()) / X_train.std()
X_test = (X_test - X_test.mean()) / X_test.std()

clf = LogisticRegression()
clf.fit(X_train, y_train)

predictions = clf.predict(X_test)
score = clf.score(X_test, y_test)

cm = metrics.confusion_matrix(y_test, predictions)
print(classification_report(y_test, predictions))

sns.heatmap(cm, annot=True)
plt.show()

In [ ]:
train_data = pd.read_csv('./knights/Train_knight.csv')

X = train_data.drop(['knight'], axis=1).values
y = train_data['knight'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

X_train = (X_train - X_train.mean()) / X_train.std()
X_test = (X_test - X_test.mean()) / X_test.std()

pca = PCA(n_components=2)
X_train = pca.fit_transform(X_train)
X_test = pca.fit_transform(X_test)

clf1 = tree.DecisionTreeClassifier()
clf2 = LogisticRegression()
clf3 = neighbors.KNeighborsClassifier(n_neighbors=30)

clf1 = clf1.fit(X_train, y_train)
clf1_predictions = clf1.predict(X_test)

clf2.fit(X_train, y_train)
clf2_predictions = clf2.predict(X_test)

clf3.fit(X_train, y_train)
clf3_predictions = clf3.predict(X_test)

eclf = VotingClassifier(estimators=[('dt', clf1), ('lr', clf2), ('knn', clf3)], voting='hard')
eclf.fit(X_train, y_train)

eclf_predictions = eclf.predict(X_test)
print(classification_report(y_test, eclf_predictions))
score = eclf.score(X_test, y_test)
print(score)

In [ ]:
test_data = pd.read_csv('./knights/Test_knight.csv')
X_test = test_data.values
pca = PCA(n_components=2)
X_test = pca.fit_transform(X_test)
predictions = eclf.predict(X_test)
np.savetxt('./data/processed/Voting.txt', predictions, fmt='%s')